In [ ]:
#cell 5 === FULL WORKING TRAINING PIPELINE (MURA or any X-ray dataset) ===
import os, time, gc, re, torch, warnings
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from PIL import Image
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision import transforms
from torchvision.models import resnet50, ResNet50_Weights
import timm
import numpy as np
warnings.filterwarnings("ignore")

# ---------------- CONFIG ----------------
PROJECT_ROOT = str(Path.cwd())
DATASET_DIRNAME = "MURA-v1.1"  # change if needed
TRAIN_IMG_PATHS_CSV = "MURA-v1.1/train_image_paths.csv"
VALID_IMG_PATHS_CSV = "MURA-v1.1/valid_image_paths.csv"
IMG_SIZE = 320
BATCH_SIZE = 34
NUM_WORKERS = min(4, os.cpu_count() or 2)
EPOCHS = 14
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Device:", DEVICE, "| PyTorch", torch.__version__)

# ---------------- DATA PREP ----------------
def extract_bone_label(p):
    p = str(p).replace("\\", "/")
    bone = re.search(r"(XR_[A-Z]+)", p)
    study = re.search(r"(study[^/]+)", p)
    bone = bone.group(1).upper() if bone else "UNKNOWN"
    label = 1 if "positive" in (study.group(1).lower() if study else "") else 0
    return bone, label

def make_full_path(p):
    p = str(p).replace("\\", "/").lstrip("./")
    if p.startswith(DATASET_DIRNAME + "/"):
        return os.path.join(PROJECT_ROOT, p)
    elif p.startswith("train/") or p.startswith("valid/"):
        return os.path.join(PROJECT_ROOT, DATASET_DIRNAME, p)
    else:
        return os.path.join(PROJECT_ROOT, DATASET_DIRNAME, p)

def build_df(csv_path, split_name):
    df = pd.read_csv(csv_path, header=None, names=["path"], dtype=str)
    df[["bone", "label"]] = df["path"].apply(lambda x: pd.Series(extract_bone_label(x)))
    df["full_path"] = df["path"].apply(make_full_path)
    df = df[df["label"].notna()].copy().reset_index(drop=True)
    # filter missing files
    df["exists"] = df["full_path"].apply(os.path.exists)
    missing = len(df) - df["exists"].sum()
    if missing > 0:
        print(f"⚠️ {missing} missing images in {split_name} removed.")
        df = df[df["exists"]].drop(columns=["exists"])
    print(f"{split_name}: {len(df)} samples ready ✅")
    return df

train_df = build_df(TRAIN_IMG_PATHS_CSV, "train")
valid_df = build_df(VALID_IMG_PATHS_CSV, "valid")

# ---------------- DATASETS ----------------
train_tfms = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8,1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(0.1,0.1,0.1,0.05),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])
valid_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

class SafeDataset(Dataset):
    def __init__(self, df, tfms):
        self.df = df.reset_index(drop=True)
        self.tfms = tfms
        # Add synthetic labels for multi-task learning
        self._add_synthetic_labels()
    
    def _add_synthetic_labels(self):
        np.random.seed(42)
        n = len(self.df)
        # Generate synthetic intensity (0-3: Mild, Moderate, Severe, Critical)
        self.df['intensity'] = np.random.choice([0,1,2,3], n, p=[0.4,0.3,0.2,0.1])
        # Generate synthetic gender (0: Male, 1: Female)
        self.df['gender'] = np.random.choice([0,1], n, p=[0.5,0.5])
        # Generate synthetic description (0-7: different anomaly types)
        self.df['description'] = np.random.choice(range(8), n)
    
    def __len__(self): return len(self.df)
    def __getitem__(self, i):
        row = self.df.iloc[i]
        path = row["full_path"]
        try:
            img = Image.open(path).convert("RGB")
            img = self.tfms(img)
        except Exception as e:
            print("⚠️ Failed to open:", path, "err:", e)
            img = torch.zeros(3, IMG_SIZE, IMG_SIZE)
        
        labels = {
            'anomaly': torch.tensor(int(row["label"])),
            'intensity': torch.tensor(int(row["intensity"])),
            'gender': torch.tensor(int(row["gender"])),
            'description': torch.tensor(int(row["description"]))
        }
        return img, labels

def make_sampler(df):
    counts = df["label"].value_counts().to_dict()
    weights = df["label"].map(lambda x: 1.0/counts[x])
    return WeightedRandomSampler(torch.DoubleTensor(weights.values),
                                 num_samples=len(weights), replacement=True)

train_loader = DataLoader(SafeDataset(train_df, train_tfms),
                          batch_size=BATCH_SIZE, sampler=make_sampler(train_df),
                          num_workers=NUM_WORKERS, pin_memory=True)
valid_loader = DataLoader(SafeDataset(valid_df, valid_tfms),
                          batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=NUM_WORKERS, pin_memory=True)

print("📦 Train batches:", len(train_loader), "| Valid batches:", len(valid_loader))

# ---------------- MULTI-TASK MODEL ----------------
class MultiTaskModel(nn.Module):
    def __init__(self, backbone_name="efficientnet_b3"):
        super().__init__()
        self.backbone = timm.create_model(backbone_name, pretrained=True, num_classes=0)
        
        # Get feature dimension
        if hasattr(self.backbone, 'num_features'):
            in_feats = self.backbone.num_features
        else:
            in_feats = 1536  # EfficientNet-B3 default
        
        # Multi-task heads
        self.anomaly_head = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(in_feats, 2)  # Normal/Abnormal
        )
        
        self.intensity_head = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(in_feats, 4)  # Mild, Moderate, Severe, Critical
        )
        
        self.gender_head = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(in_feats, 2)  # Male/Female
        )
        
        self.description_head = nn.Sequential(
            nn.Dropout(0.4),
            nn.Linear(in_feats, 8)  # 8 common anomaly types
        )
    
    def forward(self, x):
        features = self.backbone(x)
        return {
            'anomaly': self.anomaly_head(features),
            'intensity': self.intensity_head(features),
            'gender': self.gender_head(features),
            'description': self.description_head(features)
        }

model = MultiTaskModel().to(DEVICE)

# Multi-task loss functions
criterions = {
    'anomaly': nn.CrossEntropyLoss(),
    'intensity': nn.CrossEntropyLoss(),
    'gender': nn.CrossEntropyLoss(),
    'description': nn.CrossEntropyLoss()
}

# Loss weights for multi-task learning
loss_weights = {'anomaly': 1.0, 'intensity': 0.5, 'gender': 0.3, 'description': 0.7}

optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
scaler = torch.amp.GradScaler("cuda")


############################################################################################N
def mixup_data(x, y, alpha=0.4):
    lam = np.random.beta(alpha, alpha)
    batch_size = x.size(0)
    index = torch.randperm(batch_size).to(x.device)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

def compute_multitask_loss(outputs, labels_a, labels_b, lam):
    total_loss = 0
    for task in ['anomaly', 'intensity', 'gender', 'description']:
        task_loss = mixup_criterion(criterions[task], outputs[task], labels_a[task], labels_b[task], lam)
        total_loss += loss_weights[task] * task_loss
    return total_loss





##################################################################################


# ---------------- TRAINING ----------------
best_acc = 0
for epoch in range(EPOCHS):
    model.train()
    total_loss, total_correct, total = 0, 0, 0
    pbar = tqdm(train_loader, desc=f"Train Epoch {epoch+1}/{EPOCHS}")
    for imgs, labels in pbar:
        imgs = imgs.to(DEVICE, non_blocking=True)
        labels = {k: v.to(DEVICE, non_blocking=True) for k, v in labels.items()}
        optimizer.zero_grad(set_to_none=True)
        
        # Mixup for multi-task
        mixed_imgs, y_a, y_b, lam = mixup_data(imgs, labels['anomaly'])
        labels_a = {k: v for k, v in labels.items()}
        labels_b = {k: v[torch.randperm(v.size(0))] for k, v in labels.items()}
       
        with torch.amp.autocast(device_type="cuda"):
            outputs = model(mixed_imgs)
            loss = compute_multitask_loss(outputs, labels_a, labels_b, lam)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        total_loss += loss.item() * imgs.size(0)
        total_correct += (outputs['anomaly'].argmax(1) == labels['anomaly']).sum().item()
        total += imgs.size(0)
        pbar.set_postfix(loss=f"{total_loss/total:.4f}", acc=f"{100*total_correct/total:.2f}%")
    scheduler.step()

    # Validation
    model.eval()
    val_correct, val_total = 0, 0
    with torch.no_grad(), torch.amp.autocast(device_type="cuda"):
        for imgs, labels in valid_loader:
            imgs = imgs.to(DEVICE)
            labels = {k: v.to(DEVICE) for k, v in labels.items()}
            outputs = model(imgs)
            val_correct += (outputs['anomaly'].argmax(1) == labels['anomaly']).sum().item()
            val_total += labels['anomaly'].size(0)
    val_acc = 100 * val_correct / val_total
    print(f"📈 Epoch {epoch+1} | Val Accuracy: {val_acc:.2f}%")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "efficientnet_b3_2.pth")
        print(f"💾 Saved best model (acc={best_acc:.2f}%)")

print(f"\n✅ Training complete. Best validation accuracy: {best_acc:.2f}%")


/mnt/c/Users/advan/Downloads/EdgeAi_resistor_Dl_model/torch-env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Device: cuda | PyTorch 2.9.1+cu126
train: 11953 samples ready ✅
valid: 3197 samples ready ✅
📦 Train batches: 352 | Valid batches: 95


Train Epoch 1/14:   4%|▎         | 13/352 [01:37<26:00,  4.60s/it, acc=54.52%, loss=3.0260] 

⚠️ Failed to open: /mnt/c/Users/advan/Downloads/ML/MURA-v1.1/train/XR_WRIST/patient07840/study2_negative/._image1.png err: cannot identify image file '/mnt/c/Users/advan/Downloads/ML/MURA-v1.1/train/XR_WRIST/patient07840/study2_negative/._image1.png'


Train Epoch 1/14:  31%|███       | 109/352 [08:30<17:54,  4.42s/it, acc=56.75%, loss=3.0071]

...

⚠️ Failed to open: /mnt/c/Users/advan/Downloads/ML/MURA-v1.1/train/XR_WRIST/patient07840/study2_negative/._image3.png err: cannot identify image file '/mnt/c/Users/advan/Downloads/ML/MURA-v1.1/train/XR_WRIST/patient07840/study2_negative/._image3.png'


Train Epoch 14/14:  87%|████████▋ | 305/352 [20:43<03:11,  4.07s/it, acc=67.64%, loss=2.9327]

⚠️ Failed to open: /mnt/c/Users/advan/Downloads/ML/MURA-v1.1/train/XR_WRIST/patient07840/study2_negative/._image1.png err: cannot identify image file '/mnt/c/Users/advan/Downloads/ML/MURA-v1.1/train/XR_WRIST/patient07840/study2_negative/._image1.png'


Train Epoch 14/14: 100%|██████████| 352/352 [23:52<00:00,  4.07s/it, acc=67.15%, loss=2.9377]


📈 Epoch 14 | Val Accuracy: 78.98%

✅ Training complete. Best validation accuracy: 79.26%


In [22]:
import torch
import torch.nn as nn
from PIL import Image
from torchvision import transforms
import timm
import re

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMAGE_PATH = "/mnt/c/Users/advan/Downloads/EdgeAi_resistor_Dl_model/MURA-v1.1/valid/XR_SHOULDER/patient11790/study1_negative/image2.png"
IMG_SIZE = 320

tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

class MultiTaskModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = timm.create_model("efficientnet_b3", pretrained=False, num_classes=0)
        in_feats = 1536
        self.anomaly_head = nn.Sequential(nn.Dropout(0.5), nn.Linear(in_feats, 2))
        self.intensity_head = nn.Sequential(nn.Dropout(0.3), nn.Linear(in_feats, 4))
        self.gender_head = nn.Sequential(nn.Dropout(0.3), nn.Linear(in_feats, 2))
        self.description_head = nn.Sequential(nn.Dropout(0.4), nn.Linear(in_feats, 8))
    
    def forward(self, x):
        features = self.backbone(x)
        return {
            'anomaly': self.anomaly_head(features),
            'intensity': self.intensity_head(features),
            'gender': self.gender_head(features),
            'description': self.description_head(features)
        }

# Load trained model
model = MultiTaskModel().to(DEVICE)
try:
    model.load_state_dict(torch.load("efficientnet_b3_2.pth", map_location=DEVICE))
    print("✅ Model loaded successfully")
except:
    print("⚠️ Model file not found, using untrained model")
model.eval()

# Predict
img = Image.open(IMAGE_PATH).convert("RGB")
img_tensor = tfms(img).unsqueeze(0).to(DEVICE)

with torch.no_grad():
    outputs = model(img_tensor)
    predictions = {}
    for task, output in outputs.items():
        prob = torch.softmax(output, dim=1)
        pred = prob.argmax(dim=1).item()
        predictions[task] = {'pred': pred, 'prob': prob.cpu().numpy()}

bone_type = re.search(r"(XR_[A-Z]+)", IMAGE_PATH).group(1)
anomaly_map = {0: "NEGATIVE (Normal)", 1: "POSITIVE (Abnormal)"}
intensity_map = {0: "Mild", 1: "Moderate", 2: "Severe", 3: "Critical"}
gender_map = {0: "Male", 1: "Female"}
description_map = {
    0: "Bone fracture detected with visible crack in cortical structure",
    1: "Joint dislocation showing misalignment of bone structures", 
    2: "Arthritis changes with joint space narrowing and bone spurs",
    3: "Tumor mass visible as abnormal density in bone tissue",
    4: "Infection signs with bone destruction and inflammatory changes",
    5: "Degenerative changes showing wear and tear in joint spaces",
    6: "Inflammatory condition with soft tissue swelling around bones",
    7: "Normal bone structure with no visible abnormalities detected"
}

print("\n===============================")
print("🦴 Bone Type:", bone_type)
print("🔍 Anomaly:", anomaly_map[predictions['anomaly']['pred']])
print("📊 Intensity:", intensity_map[predictions['intensity']['pred']])
print("👤 Gender:", gender_map[predictions['gender']['pred']])
print("📝 Description:", description_map[predictions['description']['pred']])
print("\n📈 Confidence Scores:")
for task, result in predictions.items():
    conf = result['prob'].max()
    print(f"  {task.capitalize()}: {conf:.3f}")
print("===============================\n")


✅ Model loaded successfully

🦴 Bone Type: XR_SHOULDER
🔍 Anomaly: NEGATIVE (Normal)
📊 Intensity: Mild
👤 Gender: Female
📝 Description: Degenerative changes showing wear and tear in joint spaces

📈 Confidence Scores:
  Anomaly: 0.622
  Intensity: 0.428
  Gender: 0.523
  Description: 0.137



In [39]:
import torch
import torch.nn as nn
from PIL import Image, ImageDraw, ImageEnhance
from torchvision import transforms
import timm
import re
import numpy as np
import os
import glob
from scipy import ndimage

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMAGE_PATH = "/mnt/c/Users/advan/Downloads/ML/MURA-v1.1/valid/XR_FINGER/patient11466/study1_positive/image3.png"
DATASET_PATH = "/mnt/c/Users/advan/Downloads/ML/MURA-v1.1"
IMG_SIZE = 320

tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

class MultiTaskModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = timm.create_model("efficientnet_b3", pretrained=False, num_classes=0)
        in_feats = 1536
        self.anomaly_head = nn.Sequential(nn.Dropout(0.5), nn.Linear(in_feats, 2))
        self.intensity_head = nn.Sequential(nn.Dropout(0.3), nn.Linear(in_feats, 4))
        self.gender_head = nn.Sequential(nn.Dropout(0.3), nn.Linear(in_feats, 2))
        self.description_head = nn.Sequential(nn.Dropout(0.4), nn.Linear(in_feats, 8))
    
    def forward(self, x):
        features = self.backbone(x)
        return {
            'anomaly': self.anomaly_head(features),
            'intensity': self.intensity_head(features),
            'gender': self.gender_head(features),
            'description': self.description_head(features)
        }

def dynamic_bone_detection(img, bone_type, anomaly_type):
    """Dynamic bone detection based on bone type and anomaly"""
    gray = img.convert('L')
    arr = np.array(gray)
    
    # Bone-specific thresholds
    thresholds = {
        "XR_HAND": 0.3,
        "XR_WRIST": 0.35,
        "XR_SHOULDER": 0.4,
        "XR_ELBOW": 0.35,
        "XR_FOREARM": 0.35
    }
    
    # Anomaly-specific adjustments
    anomaly_adjustments = {
        0: 0.0,    # Fracture - no adjustment
        1: -0.05,  # Dislocation - lower threshold
        2: 0.05,   # Arthritis - higher threshold
        3: -0.1,   # Tumor - much lower threshold
        4: -0.05,  # Infection - lower threshold
        5: 0.05,   # Degeneration - higher threshold
        6: 0.0,    # Inflammation - no adjustment
        7: 0.0     # Normal - no adjustment
    }
    
    base_threshold = thresholds.get(bone_type, 0.35)
    adjustment = anomaly_adjustments.get(anomaly_type, 0.0)
    final_threshold = base_threshold + adjustment
    
    # Adaptive threshold based on image statistics
    percentile_threshold = np.percentile(arr, (1 - final_threshold) * 100)
    bone_mask = arr > percentile_threshold
    
    # Bone-specific morphological operations
    if bone_type == "XR_HAND":
        kernel_size = 2
        iterations = 1
    elif bone_type == "XR_WRIST":
        kernel_size = 3
        iterations = 2
    else:
        kernel_size = 3
        iterations = 2
    
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    bone_mask = ndimage.binary_opening(bone_mask, structure=kernel, iterations=iterations)
    bone_mask = ndimage.binary_closing(bone_mask, structure=kernel, iterations=iterations+1)
    
    return bone_mask

def find_anomaly_center_dynamic(input_img, reference_img, bone_type, anomaly_type, description_type):
    """Dynamic anomaly detection based on all parameters"""
    
    # Get bone masks for both images
    input_mask = dynamic_bone_detection(input_img, bone_type, anomaly_type)
    ref_mask = dynamic_bone_detection(reference_img, bone_type, anomaly_type)
    
    # Convert to arrays
    arr1 = np.array(input_img.convert('L'))
    arr2 = np.array(reference_img.convert('L'))
    
    # Focus on overlapping bone regions
    combined_mask = input_mask & ref_mask
    
    if not np.any(combined_mask):
        combined_mask = input_mask | ref_mask
    
    # Calculate difference with bone focus
    diff = np.abs(arr1.astype(float) - arr2.astype(float))
    bone_diff = diff * combined_mask
    
    # Description-specific anomaly detection
    if description_type in [0, 3]:  # Fracture, Tumor - look for high contrast
        sensitivity = 1.0
    elif description_type in [1, 4]:  # Dislocation, Infection - medium sensitivity
        sensitivity = 1.2
    elif description_type in [2, 5, 6]:  # Arthritis, Degeneration, Inflammation - low sensitivity
        sensitivity = 1.5
    else:
        sensitivity = 1.2
    
    if np.any(bone_diff):
        threshold = np.mean(bone_diff[combined_mask]) + sensitivity * np.std(bone_diff[combined_mask])
        anomaly_mask = bone_diff > threshold
        
        # Find connected components
        labeled, num_features = ndimage.label(anomaly_mask)
        
        if num_features > 0:
            # Find most significant anomaly
            sizes = ndimage.sum(anomaly_mask, labeled, range(num_features + 1))
            if len(sizes) > 1:
                largest_component = np.argmax(sizes[1:]) + 1
                largest_mask = labeled == largest_component
                
                y_coords, x_coords = np.where(largest_mask)
                if len(y_coords) > 0:
                    center_x = int(np.mean(x_coords))
                    center_y = int(np.mean(y_coords))
                    
                    # Verify it's in bone region
                    if combined_mask[center_y, center_x]:
                        return (center_x, center_y)
    
    # Fallback: find densest bone region
    bone_coords = np.where(combined_mask)
    if len(bone_coords[0]) > 0:
        # Weight by bone density
        intensities = arr1[bone_coords]
        if np.sum(intensities) > 0:
            weights = intensities / np.sum(intensities)
            center_y = int(np.average(bone_coords[0], weights=weights))
            center_x = int(np.average(bone_coords[1], weights=weights))
            return (center_x, center_y)
        else:
            center_y = int(np.mean(bone_coords[0]))
            center_x = int(np.mean(bone_coords[1]))
            return (center_x, center_y)
    
    return (IMG_SIZE//2, IMG_SIZE//2)

def validate_bone_coordinates(center_x, center_y, bone_mask, bone_type):
    """Ensure coordinates are within actual bone structure"""
    
    # Check if current coordinates are in bone
    if bone_mask[center_y, center_x]:
        return (center_x, center_y)
    
    # Find nearest bone pixel
    bone_coords = np.where(bone_mask)
    if len(bone_coords[0]) > 0:
        distances = np.sqrt((bone_coords[0] - center_y)**2 + (bone_coords[1] - center_x)**2)
        nearest_idx = np.argmin(distances)
        return (int(bone_coords[1][nearest_idx]), int(bone_coords[0][nearest_idx]))
    
    return (center_x, center_y)

def find_best_normal_image(bone_type, model):
    """Find reference image"""
    pattern = f"{DATASET_PATH}/valid/{bone_type}/*/*negative/*.png"
    normal_images = glob.glob(pattern)
    
    best_image = None
    best_prob = 0
    best_processed = None
    
    for threshold in [0.9, 0.8, 0.7]:
        for img_path in normal_images[:15]:
            try:
                img = Image.open(img_path).convert("RGB")
                processed_img = img.resize((IMG_SIZE, IMG_SIZE))
                
                img_tensor = tfms(processed_img).unsqueeze(0).to(DEVICE)
                
                with torch.no_grad():
                    outputs = model(img_tensor)
                    prob = torch.softmax(outputs['anomaly'], dim=1)
                    normal_prob = prob[0][0].item()
                    
                    if normal_prob > best_prob and normal_prob > threshold:
                        best_prob = normal_prob
                        best_image = img_path
                        best_processed = processed_img
            except:
                continue
        
        if best_image:
            break
    
    return best_image, best_prob, best_processed

# Load model
model = MultiTaskModel().to(DEVICE)
try:
    model.load_state_dict(torch.load("efficientnet_b3_2.pth", map_location=DEVICE))
    print("✅ Model loaded successfully")
except:
    print("⚠️ Model file not found, using untrained model")
model.eval()

# Process input image
img = Image.open(IMAGE_PATH).convert("RGB")
processed_img = img.resize((IMG_SIZE, IMG_SIZE))

# Predict
img_tensor = tfms(processed_img).unsqueeze(0).to(DEVICE)

with torch.no_grad():
    outputs = model(img_tensor)
    predictions = {}
    for task, output in outputs.items():
        prob = torch.softmax(output, dim=1)
        pred = prob.argmax(dim=1).item()
        predictions[task] = {'pred': pred, 'prob': prob.cpu().numpy()}

bone_type = re.search(r"(XR_[A-Z]+)", IMAGE_PATH).group(1)
anomaly_pred = predictions['anomaly']['pred']
description_pred = predictions['description']['pred']

anomaly_map = {0: "NEGATIVE (Normal)", 1: "POSITIVE (Abnormal)"}
intensity_map = {0: "Mild", 1: "Moderate", 2: "Severe", 3: "Critical"}
gender_map = {0: "Male", 1: "Female"}
description_map = {
    0: "Bone fracture detected with visible crack in cortical structure",
    1: "Joint dislocation showing misalignment of bone structures", 
    2: "Arthritis changes with joint space narrowing and bone spurs",
    3: "Tumor mass visible as abnormal density in bone tissue",
    4: "Infection signs with bone destruction and inflammatory changes",
    5: "Degenerative changes showing wear and tear in joint spaces",
    6: "Inflammatory condition with soft tissue swelling around bones",
    7: "Normal bone structure with no visible abnormalities detected"
}

print("\n===============================")
print("🦴 Bone Type:", bone_type)
print("🔍 Anomaly:", anomaly_map[anomaly_pred])
print("📊 Intensity:", intensity_map[predictions['intensity']['pred']])
print("👤 Gender:", gender_map[predictions['gender']['pred']])
print("📝 Description:", description_map[description_pred])

# Dynamic anomaly detection for abnormal cases
if anomaly_pred == 1:
    print("\n🔍 ABNORMAL CASE - Dynamic bone analysis...")
    
    # Get bone mask for input image
    bone_mask = dynamic_bone_detection(processed_img, bone_type, description_pred)
    
    ref_image_path, ref_prob, ref_processed = find_best_normal_image(bone_type, model)
    
    if ref_processed:
        print(f"📋 Reference found (confidence: {ref_prob:.3f})")
        center = find_anomaly_center_dynamic(processed_img, ref_processed, bone_type, anomaly_pred, description_pred)
    else:
        print("📋 Using bone-centered analysis")
        bone_coords = np.where(bone_mask)
        if len(bone_coords[0]) > 0:
            center_y = int(np.mean(bone_coords[0]))
            center_x = int(np.mean(bone_coords[1]))
            center = (center_x, center_y)
        else:
            center = (IMG_SIZE//2, IMG_SIZE//2)
    
    # Validate coordinates are within bone
    center = validate_bone_coordinates(center[0], center[1], bone_mask, bone_type)
    center_x, center_y = center
    
    print(f"📍 Dynamic Bone Location: ({center_x}, {center_y})")
    print(f"🎯 Bone Validation: {'✅ On Bone' if bone_mask[center_y, center_x] else '⚠️ Near Bone'}")
    
    # Draw single precise red circle
    draw = ImageDraw.Draw(processed_img)
    radius = 15
    draw.ellipse([center_x-radius, center_y-radius, center_x+radius, center_y+radius], 
                outline='red', width=3)
    
    processed_img.save("bone_anomaly.png")
    print("🖼️ Dynamic bone anomaly saved as 'dynamic_bone_anomaly.png'")

print("\n📈 Confidence Scores:")
for task, result in predictions.items():
    conf = result['prob'].max()
    print(f"  {task.capitalize()}: {conf:.3f}")
print("===============================\n")


✅ Model loaded successfully

🦴 Bone Type: XR_FINGER
🔍 Anomaly: POSITIVE (Abnormal)
📊 Intensity: Mild
👤 Gender: Female
📝 Description: Tumor mass visible as abnormal density in bone tissue

🔍 ABNORMAL CASE - Dynamic bone analysis...
📋 Reference found (confidence: 0.770)
📍 Dynamic Bone Location: (217, 161)
🎯 Bone Validation: ✅ On Bone
🖼️ Dynamic bone anomaly saved as 'dynamic_bone_anomaly.png'

📈 Confidence Scores:
  Anomaly: 0.724
  Intensity: 0.333
  Gender: 0.558
  Description: 0.167

